# PEC correction

Start by importing MagmaPEC and MagmaPandas and any other packages you want to use. Here we also import Pandas for importing pressure data. For details on the use of MagmaPandas, please see it's [documentation](https://magmapandas.readthedocs.io/en/latest/).

In [1]:
import MagmaPEC as mpc
import MagmaPandas as mp

import pandas as pd

Import your melt inclusion and olivine data as magmapandas dataframes. Expected units are wt.%, and in the column headers only the first letter of each element should be capitalized. Iron is expected to be expressed as FeO (total), with column name 'FeO'.

In [2]:
melt_file = "./data/melt.csv"
olivine_file = "./data/olivine.csv"

melt = mp.read_melt(melt_file, index_col=["name"], units="wt. %")
olivine = mp.read_olivine(olivine_file, index_col=["name"], units="wt. %")

Here's an example of the expected column header formatting:

In [3]:
print(melt.columns.values)

['SiO2' 'Al2O3' 'MgO' 'CaO' 'FeO' 'Na2O' 'K2O' 'MnO' 'TiO2' 'P2O5' 'Cr2O3'
 'CO2' 'H2O' 'F' 'S' 'Cl']


If you have data for internal inclusion pressures (in bars) you can also import those. The data need to be list-like (e.g. list, numpy array, pandas series) and if you use a pandas series you need to make sure the index matches those of the melt and olivine dataframes.

The pressures we use here were calculated previously with CO2-H2O solubility models. CO2 from the glass only was used, i.e. not from the bubble, and resulting pressures are final pressures inside the inclusions, at the end of post-entrapment crystallization.

In reality, pressures are not fixed during post-entrapment crystallisation. Olivine has a lower specific volume than silicate melt, so as inclusions crystallize olivine, their internal pressures decrease. However, many melt inclusions are trapped at high pressure (>5 kbar) and compared to that, PEC-driven pressure decreases are relatively minor (<2 kbar). Moreover, modelling pressure changes during post-entrapment crystallization is complicated and requires tight thermodynamic constraints. MagmaPEC therefore runs at fixed pressures. 

Suitable pressure options are final, post-PEC, pressures or an approximations of entrapment pressures. Entrapment pressures can be approximated by calculating CO2 saturation pressures of uncorrected inclusions with their total CO2 budgets (glass + bubble + carbonates), or by running PEC corrections iteratively. Note that the former results in overestimated saturation pressures as PEC corrections dilute inclusion CO2 concentrations. Modelled PEC extents increase with pressure. Post-PEC pressures will therefore yield slightly underestimated PEC extents, and entrapment pressures slightly overestimated PEC extents.

In [4]:
pressure_file ="./data/pressure.csv"

pressure = pd.read_csv(pressure_file, index_col = ["name"]).squeeze()

pressure

name
PI032-04-01    4104.410589
PI032-04-02    4231.966408
PI041-02-02    1956.244091
PI041-03-01    3548.843351
PI041-03-03    3500.514426
PI041-05-04    1386.274577
PI041-05-06    2512.013195
PI041-07-01    3668.361134
PI041-07-02    2449.116051
PI052-01-02    1404.218415
Name: IaconoMarziano, dtype: float64

Alternatively, if you have melt CO<sub>2</sub> (and H<sub>2</sub>O) data, you can use MagmaPandas to calculate volatile saturation pressures. First calculate temperature with some initial pressure and then iteratively calculate pressure and temperature. See the [MagmaPandas documentation](https://magmapandas.readthedocs.io/en/latest/notebooks/melt_basics.html) for more details.

The melt compositions in this example have been corrected for CO2 bubbles and the pressures calculated here are therefore overestimated approximations of inclusion entrapment pressures (see above).

In the rest of this notebook we do not use these modelled pressures, and instead use the above imported post-PEC pressures.

In [5]:
temperature = melt.temperature(P_bar = 5e3)
pressure_calc = melt.volatile_saturation_pressure(T_K=temperature)

while True:
    temperature = melt.temperature(P_bar=pressure_calc)
    pressure_calc_new = melt.volatile_saturation_pressure(T_K=temperature)
    dP = (pressure_calc_new - pressure_calc) / pressure_calc_new # calculate percentage change
    pressure_calc = pressure_calc_new.copy()
    if (dP < 0.01).all():
        # break when all pressure have converged within 1% of previous values.
        break

pressure_calc

Saturation pressure...    |█████████████████████████| 100% [10/10] in 1.3s 
Saturation pressure...    |█████████████████████████| 100% [10/10] in 1.4s 


name
PI032-04-01    6527.085981
PI032-04-02    6828.021519
PI041-02-02    4802.872035
PI041-03-01    6958.250380
PI041-03-03    6826.596776
PI041-05-04    4013.630505
PI041-05-06    5269.481653
PI041-07-01    4969.100468
PI041-07-02    4175.590767
PI052-01-02    2388.318094
dtype: float64

Here we will also use a whole-rock dataset to set up a model for predicting the initial FeO contents of our melt inclusions. For explanation of this model, follow the [Initial FeO prediction](https://magmapec.readthedocs.io/en/latest/notebooks/FeOi.html#) example.

In [6]:
wholerock_file = "./data/wholerock.csv"

wholerock = mp.read_melt(wholerock_file, index_col=["name"])

Let's begin by setting up the model to predict the initial FeO content of our melt inclusions according to the previous [example](https://magmapec.readthedocs.io/en/latest/notebooks/FeOi.html#). Here, the FeO prediction model is based on TiO<sub>2</sub>, Al<sub>2</sub>O<sub>3</sub> and CaO, which we can check by accessing their coefficients:

In [7]:
x = wholerock.drop(columns=["FeO"])
FeOi_predict = mpc.FeOi_prediction(x=x, FeO=wholerock["FeO"])

do_not_use = ["MnO", "P2O5", "Cr2O3", "total"]

model_fits = FeOi_predict.calculate_model_fits(exclude=do_not_use)
FeOi_predict.select_predictors(idx=3)

In [8]:
FeOi_predict.coefficients

intercept    11.585125
TiO2          1.463365
Al2O3        -0.230722
CaO          -0.169262
dtype: float64

To use this model during PEC correction, we need to store this model as a callable function, which we can do with the *model* attribute:

In [9]:
FeO_model = FeOi_predict.model

Let's first quickly test the model by predicting FeO contents for our uncorrected melt compositions:

In [10]:
FeO_model(melt)

name
PI032-04-01    10.196984
PI032-04-02    10.265411
PI041-02-02    10.221895
PI041-03-01    10.614069
PI041-03-03    10.716794
PI041-05-04     9.420381
PI041-05-06    11.476340
PI041-07-01    11.608516
PI041-07-02    11.451018
PI052-01-02     8.722012
dtype: float32

This all looks good, so let continue with setting up the PEC correction model.

First, preview the melt and olivine data to check everything looks ok:

In [11]:
melt.head()

,SiO2,Al2O3,MgO,CaO,FeO,Na2O,K2O,MnO,TiO2,P2O5,Cr2O3,CO2,H2O,F,S,Cl
name,,,,,,,,,,,,,,,,
PI032-04-01,50.028862,15.226979,5.330006,10.522019,8.59454,3.914280,0.730798,0.126266,2.669216,0.303542,NaN,0.658252,1.508379,0.082590,0.152337,0.035565
PI032-04-02,49.558861,16.041588,5.117479,10.266067,8.25471,3.782408,0.997826,0.129969,2.814807,0.356900,NaN,0.713472,1.412274,0.089010,0.175511,0.047265
PI041-02-02,49.112045,16.972038,4.864153,9.195718,10.07540,3.793512,1.079184,0.153456,2.807967,0.561756,NaN,0.464520,0.656341,0.047715,0.068108,0.021185
PI041-03-01,47.098515,17.448515,4.650640,12.154110,7.95402,3.707263,1.268282,0.101370,3.493271,0.611504,NaN,0.883720,0.324968,0.088070,0.096228,0.059958
PI041-03-03,46.478870,17.637102,4.762752,12.364033,7.84027,3.789446,1.294603,0.077221,3.617484,0.574248,NaN,0.910470,0.344988,0.090605,0.088145,0.061584


In [12]:
olivine.head()

,SiO2,FeO,MgO,NiO,MnO,Al2O3,CaO,total
name,,,,,,,,
PI032-04-01,38.204800,15.981400,44.194000,0.172665,0.234256,0.005229,0.240552,99.032906
PI032-04-02,38.638500,15.898400,43.467400,0.188024,0.219599,-0.007310,0.234622,98.639240
PI041-02-02,37.270100,20.815901,41.015099,0.104744,0.293681,0.016547,0.214939,99.731000
PI041-03-01,38.795799,15.469300,44.750999,0.180198,0.212197,0.031333,0.259919,99.699745
PI041-03-03,38.701900,15.782500,44.920799,0.164372,0.214836,0.037263,0.275049,100.096720


Make sure that each row in *melt* and *olivine* is a matching pair of melt inclusion and olivine host. Here we use the sample names stored in the indices of both dataframes:

In [13]:
print(melt.index, "\n\n", olivine.index)

Index(['PI032-04-01', 'PI032-04-02', 'PI041-02-02', 'PI041-03-01',
       'PI041-03-03', 'PI041-05-04', 'PI041-05-06', 'PI041-07-01',
       'PI041-07-02', 'PI052-01-02'],
      dtype='object', name='name') 

 Index(['PI032-04-01', 'PI032-04-02', 'PI041-02-02', 'PI041-03-01',
       'PI041-03-03', 'PI041-05-04', 'PI041-05-06', 'PI041-07-01',
       'PI041-07-02', 'PI052-01-02'],
      dtype='object', name='name')


We can verify that the indices are the same with the *equals* method:

In [14]:
melt.index.equals(olivine.index)

True

Confirm the model and PEC configurations. If you want to change models or PEC settings, follow the [configuration example](https://magmapec.readthedocs.io/en/latest/notebooks/config.html).

In [15]:
print(mpc.model_configuration)
print(mpc.PEC_configuration)


################## MagmaPandas ###################
##################################################
General settings__________________________________
fO2 buffer.....................................QFM
ΔfO2.............................................1
Melt Fe3+/Fe2+.............................sun2024
Kd Fe-Mg ol-melt........................toplis2005
Melt thermometer....................putirka2008_15
Volatile solubility model.......iaconomarziano2012
Volatile species.............................mixed
##################################################


############ Post-entrapment crystallisation ############
################### correction model ####################
Settings_________________________________________________
Fe2+ behaviour...................................buffered
Stepsize equilibration (moles)...................0.002   
Stepsize crystallisation (moles).................0.05    
Decrease factor..................................5       
FeO convergence (wt. %).......

Now we can initialise the PEC model, where we need the following data:

- **inclusions**

    inclusion major element compositions in oxide wt. % as a MagmaPandas Melt frame.

- **olivines**

    olivine major element compositions in oxide wt. % as a MagmaPandas Olivine frame.

- **P_bar**

    pressures in bar at which to run the model. You can use a fixed pressure for all inclusions, e.g. *P_bar=2e3* for 2 kbar, or indicate specific pressures per inclusion. In this example we do the latter, by passing the above imported pandas Series with internal inclusion pressures

- **FeO_target**

    Estimated initial FeO contents of melt inclusions. You can use a fixed value for all melt inclusions, e.g. FeO = 11, specific values for individual melt inclusions, stored in a pandas Series, or a predictive equation based on melt major element composition. This equation needs to be a callable function that accepts a Pandas DataFrame with melt compositions in oxide wt. % as input and return an array-like object with initial FeO contents per inclusion. In the example above we showed how to set up a function like this using MagmaPEC.

In [16]:
pec_model = mpc.PEC(inclusions=melt, olivines=olivine, P_bar=pressure, FeO_target=FeO_model)

Now we simply run the model with the *correct* method. It runs in two stages: the first stage equilibrates Fe and Mg between melt inclusions and olivine hosts through Fe-Mg cation exchange and the second stage corrects melt inclusions back to their initial FeO contents by melting or crystallising olivine.

This method returns three objects:

- corrected melt compositions as a MagmaPandas Melt frame

- PEC extents as a pandas DataFrame
    
- checks to confirm the inclusions have equilibrated and reached FeOi

In [17]:
melts_corrected, pec, checks = pec_model.correct()


Equilibrating ... |██████████████████████████████| 100% [10/10] in 3.3s 
Correcting    ... |██████████████████████████████| 100% [10/10] in 11.8s 


The `checks` dataframe shows that isothermal equilibration during stage [1], Kd equilibration in stage [2], and FeO convergence in stage [2] were successful for all inclusions. This is the expected result, but during [Monte Carlo error propagation](https://magmapec.readthedocs.io/en/latest/notebooks/pec_mc.html), extreme cases of random error sampling may yield melt-olivine pairs that cannot be equilibrated without requiring crystallising exceeding the mass of the inclusion inclusion or exchange of more Mg or Fe than the inclusion contains. If that is the case, no corrected compositions are calculated and the isothermal- of Kd-equilibration column is set to *False*.

In [18]:
checks

,isothermal_equilibration,Kd_equilibration,FeO_converge
name,,,
PI032-04-01,True,True,True
PI032-04-02,True,True,True
PI041-02-02,True,True,True
PI041-03-01,True,True,True
PI041-03-03,True,True,True
PI041-05-04,True,True,True
PI041-05-06,True,True,True
PI041-07-01,True,True,True
PI041-07-02,True,True,True


The `pec` dataframe has three columns:

- equilibration_crystallisation: 
    
    crystallisation extents during the equilibration stage.

- PE_crystallisation:

    crystallisation extents during the crystallisation stage.

- total_crystallisation:

    total amount of post-entrapment crystallisation

with all data in percent and positive values indicating post-entrapment crystallisation and negative melting

In [19]:
pec

/Users/thomas/miniforge3/envs/py310/lib/python3.10/site-packages/pandas/io/formats/format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


,equilibration_crystallisation,PE_crystallisation,total_crystallisation
name,,,
PI032-04-01,-2.589844,12.4,9.810583
PI032-04-02,-2.181641,13.0,10.817993
PI041-02-02,-0.404785,0.4,-0.004739
PI041-03-01,-1.751953,14.4,12.648291
PI041-03-03,-0.867676,13.0,12.132538
PI041-05-04,-1.602539,-2.8,-4.402173
PI041-05-06,-1.605469,3.6,1.994775
PI041-07-01,-2.976562,15.4,12.423010
PI041-07-02,-1.678711,12.8,11.121533


These crystallization extents result in the following corrected melt compositions:

In [20]:
melts_corrected

,SiO2,Al2O3,MgO,CaO,FeO,Na2O,K2O,MnO,TiO2,P2O5,Cr2O3,CO2,H2O,F,S,Cl,total
name,,,,,,,,,,,,,,,,,
PI032-04-01,48.932615,13.997643,7.685414,9.695639,10.306935,3.598133,0.671773,0.138911,2.453630,0.279025,0.000000,0.605086,1.386551,0.075919,0.140033,0.032693,100.0
PI032-04-02,48.478496,14.628181,7.549098,9.385885,10.342922,3.449319,0.909955,0.140868,2.566928,0.325471,0.000000,0.650642,1.287905,0.081172,0.160055,0.043102,100.0
PI041-02-02,49.151067,16.987807,4.818808,9.204932,10.173751,3.797024,1.080183,0.154553,2.810567,0.562276,0.000000,0.464950,0.656948,0.047759,0.068171,0.021204,100.0
PI041-03-01,45.917518,15.664650,7.210145,10.936913,10.702791,3.327537,1.138375,0.113616,3.135464,0.548870,0.000000,0.793203,0.291682,0.079049,0.086372,0.053817,100.0
PI041-03-03,45.331776,15.888064,7.027840,11.161822,10.799127,3.412885,1.165957,0.090181,3.258011,0.517185,0.000000,0.819995,0.310707,0.081601,0.079386,0.055464,100.0
PI041-05-04,47.862023,18.767805,3.607781,9.486345,9.277349,4.643880,1.622223,0.136675,2.510764,0.833209,0.000000,0.515007,0.466122,0.088609,0.122617,0.059592,100.0
PI041-05-06,46.422610,17.057541,4.662195,8.968911,11.465500,4.022334,1.424093,0.168945,3.652662,0.622665,0.000000,0.643353,0.588746,0.107968,0.128776,0.063701,100.0
PI041-07-01,45.783040,15.214560,7.317821,9.673379,11.600875,3.126866,1.270842,0.148978,3.528233,0.554192,0.000000,0.467155,1.019572,0.075009,0.163223,0.056256,100.0
PI041-07-02,45.865882,15.593893,6.879340,10.088148,11.463062,3.193845,1.361987,0.136368,3.539169,0.616881,0.000000,0.372679,0.617050,0.073673,0.140582,0.057441,100.0


Results are also stored in the pec object and can be accessed via the `inclusions` and `olivine_corrected` attributes.

In [21]:
pec_model.inclusions

,SiO2,Al2O3,MgO,CaO,FeO,Na2O,K2O,MnO,TiO2,P2O5,Cr2O3,CO2,H2O,F,S,Cl,total
name,,,,,,,,,,,,,,,,,
PI032-04-01,48.932615,13.997643,7.685414,9.695639,10.306935,3.598133,0.671773,0.138911,2.453630,0.279025,0.000000,0.605086,1.386551,0.075919,0.140033,0.032693,100.0
PI032-04-02,48.478496,14.628181,7.549098,9.385885,10.342922,3.449319,0.909955,0.140868,2.566928,0.325471,0.000000,0.650642,1.287905,0.081172,0.160055,0.043102,100.0
PI041-02-02,49.151067,16.987807,4.818808,9.204932,10.173751,3.797024,1.080183,0.154553,2.810567,0.562276,0.000000,0.464950,0.656948,0.047759,0.068171,0.021204,100.0
PI041-03-01,45.917518,15.664650,7.210145,10.936913,10.702791,3.327537,1.138375,0.113616,3.135464,0.548870,0.000000,0.793203,0.291682,0.079049,0.086372,0.053817,100.0
PI041-03-03,45.331776,15.888064,7.027840,11.161822,10.799127,3.412885,1.165957,0.090181,3.258011,0.517185,0.000000,0.819995,0.310707,0.081601,0.079386,0.055464,100.0
PI041-05-04,47.862023,18.767805,3.607781,9.486345,9.277349,4.643880,1.622223,0.136675,2.510764,0.833209,0.000000,0.515007,0.466122,0.088609,0.122617,0.059592,100.0
PI041-05-06,46.422610,17.057541,4.662195,8.968911,11.465500,4.022334,1.424093,0.168945,3.652662,0.622665,0.000000,0.643353,0.588746,0.107968,0.128776,0.063701,100.0
PI041-07-01,45.783040,15.214560,7.317821,9.673379,11.600875,3.126866,1.270842,0.148978,3.528233,0.554192,0.000000,0.467155,1.019572,0.075009,0.163223,0.056256,100.0
PI041-07-02,45.865882,15.593893,6.879340,10.088148,11.463062,3.193845,1.361987,0.136368,3.539169,0.616881,0.000000,0.372679,0.617050,0.073673,0.140582,0.057441,100.0


In [22]:
pec_model.olivine_corrected

/Users/thomas/miniforge3/envs/py310/lib/python3.10/site-packages/pandas/io/formats/format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


,equilibration_crystallisation,PE_crystallisation,total_crystallisation
name,,,
PI032-04-01,-2.589844,12.4,9.810583
PI032-04-02,-2.181641,13.0,10.817993
PI041-02-02,-0.404785,0.4,-0.004739
PI041-03-01,-1.751953,14.4,12.648291
PI041-03-03,-0.867676,13.0,12.132538
PI041-05-04,-1.602539,-2.8,-4.402173
PI041-05-06,-1.605469,3.6,1.994775
PI041-07-01,-2.976562,15.4,12.423010
PI041-07-02,-1.678711,12.8,11.121533


pressures, temperatures, ol-melt Fe-Mg Kd (modelled equilibrium and current inclusion value), melt Fe3Fe2, and fO2 of corrected inclusions can be calculated with the `get_PTX` method. Pressures can be inputted as a method argument, or calculated as entrapment pressures with H2O-CO2 solubility models if concentrations for those elements are available.

In [33]:
# with fixed pressures 
ptx_1bar = pec_model.get_PTX(P_bar=1)
ptx_1bar

,P_bar,T_K,Kd_modelled,Kd_inclusion,Fe3Fe2,fO2_Pa
name,,,,,,
PI032-04-01,1,1460.728274,0.312547,0.320435,0.188366,2.770090e-08
PI032-04-02,1,1459.155668,0.310149,0.317740,0.190227,2.652634e-08
PI041-02-02,1,1396.927228,0.301447,0.294961,0.227029,4.450087e-09
PI041-03-01,1,1462.908507,0.285660,0.293955,0.262310,2.941173e-08
PI041-03-03,1,1459.204225,0.283334,0.289899,0.267902,2.656188e-08
PI041-05-04,1,1368.972779,0.284882,0.274622,0.255115,1.901566e-09
PI041-05-06,1,1400.268876,0.292366,0.286071,0.251544,4.915851e-09
PI041-07-01,1,1459.972359,0.300747,0.305522,0.218366,2.713024e-08
PI041-07-02,1,1453.929502,0.293611,0.295200,0.245878,2.295525e-08


In [34]:
# with entrapment pressures calculated from inclusions CO2-H2O
ptx = pec_model.get_PTX()
ptx

Saturation pressure...    |█████████████████████████| 100% [10/10] in 1.6s 
Saturation pressure...    |█████████████████████████| 100% [10/10] in 1.8s 
Saturation pressure...    |█████████████████████████| 100% [10/10] in 1.8s 
Saturation pressure...    |█████████████████████████| 100% [10/10] in 1.6s 


,P_bar,T_K,Kd_modelled,Kd_inclusion,Fe3Fe2,fO2_Pa
name,,,,,,
PI032-04-01,6102.850920,1485.539772,0.322887,0.315356,0.169530,1.474123e-07
PI032-04-02,5860.066830,1484.776133,0.320281,0.313242,0.173378,1.388899e-07
PI041-02-02,4804.040525,1415.735934,0.309981,0.290466,0.208330,1.772361e-08
PI041-03-01,6445.499340,1488.145162,0.295494,0.288551,0.239105,1.667916e-07
PI041-03-03,6347.113933,1484.055604,0.293016,0.284286,0.243350,1.476052e-07
PI041-05-04,4149.754388,1385.219301,0.292260,0.269967,0.233839,6.500694e-09
PI041-05-06,5198.688741,1420.623034,0.301375,0.280899,0.228915,2.178971e-08
PI041-07-01,4743.646588,1478.544541,0.308459,0.301084,0.200671,9.823091e-08
PI041-07-02,3802.069480,1468.814490,0.299738,0.291212,0.229047,6.492697e-08
